```{contents}
```
## ReAct Agent (Reason + Act)

### What a ReAct Agent Is

A **ReAct agent** is an agentic pattern where an LLM **interleaves reasoning (“Thought”) with actions (“Act”)** to solve a task step by step.

> ReAct = **Reasoning + Acting**

Instead of answering in one shot, the model:

1. Thinks about what to do next
2. Calls a tool if needed
3. Observes the result
4. Thinks again
5. Repeats until it can answer

---

### Why ReAct Agents Exist

Single-pass LLM calls fail when:

* External data is required
* Multiple steps are needed
* Intermediate validation is required
* Decisions depend on tool outputs

ReAct agents enable:

* Multi-step problem solving
* Tool-driven reasoning
* Dynamic decision-making
* Higher accuracy on complex tasks

---

### Core ReAct Loop

```
User Question
   ↓
Thought (reasoning)
   ↓
Action (tool call)
   ↓
Observation (tool result)
   ↓
Thought
   ↓
Final Answer
```

This loop can run **multiple times**.

---

### Thought / Action / Observation Explained

### Thought

Internal reasoning step:

* What do I know?
* What do I need next?
* Which tool should I use?

> Thought is **not shown to the user** in production systems.

---

### Action

A structured tool call:

* API
* Database query
* Search
* Calculator
* RAG retriever

---

### Observation

The deterministic output returned by the tool.

---

### Simple ReAct Example (Conceptual)

**User:**
“How many Jira tickets are open, and is that above the SLA threshold?”

**Agent reasoning (simplified):**

```
Thought: I need the number of open tickets → use ticket_count tool
Action: ticket_count(source="jira")
Observation: 128
Thought: I need SLA threshold → use sla_lookup tool
Action: sla_lookup()
Observation: 100
Thought: Compare values and answer
Final Answer: Yes, open tickets exceed the SLA threshold.
```

---

### ReAct vs Tool Calling

| Aspect          | Tool Calling | ReAct Agent |
| --------------- | ------------ | ----------- |
| Reasoning steps | Single       | Multiple    |
| Tool calls      | One          | Many        |
| Decision making | Minimal      | Dynamic     |
| Complexity      | Low          | High        |

Tool calling is **one step**.
ReAct is a **loop**.

---

### ReAct vs Sequential Chain

| Aspect    | Sequential Chain | ReAct Agent |
| --------- | ---------------- | ----------- |
| Flow      | Fixed            | Dynamic     |
| Branches  | ❌                | ✅           |
| Tools     | ❌                | ✅           |
| Reasoning | ❌                | ✅           |

---

### ReAct in LangChain

LangChain provides built-in ReAct-style agents.

Core components:

* Chat model
* Tools
* Agent prompt
* Agent executor

---

### ReAct Agent Demonstration (LangChain)

#### Step 1: Define Tools



In [1]:
from langchain_classic.tools import tool

@tool
def ticket_count(source: str) -> int:
    """Return number of tickets from a source"""
    return 128



---

#### Step 2: Create the LLM

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)



Low temperature is critical for agent stability.

---

#### Step 3: Create ReAct Agent

In [5]:
from langchain_classic.agents import create_react_agent
from langchain_classic import hub

# Pull the ReAct prompt template from LangChain Hub
prompt = hub.pull("hwchase17/react")

agent = create_react_agent(
    llm=llm,
    tools=[ticket_count],
    prompt=prompt
)



LangChain injects a **ReAct-style prompt** automatically.

---

#### Step 4: Execute the Agent



In [7]:
from langchain_classic.agents import AgentExecutor

executor = AgentExecutor(
    agent=agent,
    tools=[ticket_count],
    verbose=True
)

executor.invoke(
    {"input": "How many tickets are there in Jira?"}
)




> Entering new AgentExecutor chain...
I need to find out the number of tickets in Jira. To do this, I will use the `ticket_count` function with "Jira" as the source.  
Action: ticket_count  
Action Input: "Jira"  128I now know the final answer  
Final Answer: 128

> Finished chain.


{'input': 'How many tickets are there in Jira?', 'output': '128'}



---

### What Happens Internally

1. LLM reasons about the question
2. Decides to call `ticket_count`
3. Tool executes
4. Observation is passed back
5. LLM produces final answer

---

### ReAct Prompt (Simplified Concept)

Internally, the agent follows a structure like:

```
Thought: ...
Action: tool_name(arguments)
Observation: ...
```

In production:

* Thoughts are hidden
* Only the final answer is returned

---

### ReAct + RAG (Very Common)

ReAct agents often use **retrieval as a tool**:

```
Thought: I need documentation
Action: retrieve_docs(query)
Observation: relevant chunks
Thought: Answer using docs
```

This enables **multi-hop RAG**.

---

### Failure Modes of ReAct Agents

### Infinite Loops

* Agent keeps calling tools
* Mitigation: max iterations

### Wrong Tool Choice

* Tool descriptions unclear
* Mitigation: better tool schemas

### Hallucinated Observations

* Prevented by deterministic tools

---

### Guardrails for ReAct Agents

* Low temperature
* Strict tool schemas
* Tool allowlists
* Max iteration limits
* Output validation
* Human approval for write actions

---

### When to Use ReAct Agents

* Multi-step reasoning
* Tool-heavy workflows
* Dynamic decision making
* Complex enterprise automation

---

### When NOT to Use ReAct Agents

* Simple Q&A
* Static RAG
* Latency-critical endpoints
* Deterministic workflows

---

### ReAct vs LangGraph

| Aspect            | ReAct      | LangGraph              |
| ----------------- | ---------- | ---------------------- |
| Control           | LLM-driven | Explicit state machine |
| Predictability    | Medium     | High                   |
| Complexity        | Medium     | High                   |
| Production safety | Medium     | High                   |

LangGraph is often preferred for **mission-critical systems**.

---

### Interview-Ready Summary

> “A ReAct agent interleaves reasoning and tool-based actions in a loop, allowing LLMs to solve complex tasks dynamically. It improves accuracy and flexibility but requires guardrails to be production-safe.”

---

### Rule of Thumb

* **One tool call → Tool calling**
* **Many steps → ReAct agent**
* **Strict workflows → LangGraph**
* **Enterprise safety → ReAct + guardrails**